In [1]:
import numpy as np

import tensorflow as tf
import tensorflow.lite as tflite

import preprocessing
import requests

tf.__version__

2023-11-22 23:29:36.294210: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 23:29:36.382741: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-22 23:29:36.382812: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-22 23:29:36.384587: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-22 23:29:36.396219: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2023-11-22 23:29:36.396695: I tensorflow/core/platform/cpu_feature_guard.cc:1

'2.15.0'

In [2]:
! ls -lh -g -o bees-wasps.h5

-rw-rw-r-- 1 86M лис 20 21:47 bees-wasps.h5


In [3]:
model = tf.keras.models.load_model('bees-wasps.h5')

2023-11-22 23:29:44.026004: I external/local_xla/xla/stream_executor/cuda/cuda_executor.cc:901] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2023-11-22 23:29:44.605625: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2256] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-11-22 23:29:45.147569: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2023-11-22 23:29:45.236367: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% 

## Question 1

Now convert this model from Keras to TF-Lite format.

What's the size of the **converted** model?

In [4]:
converter = tflite.TFLiteConverter.from_keras_model(model)

tflite_model = converter.convert()

with open('bees_wasps.tflite', 'wb') as f_out:
    f_out.write(tflite_model)

INFO:tensorflow:Assets written to: /tmp/tmpjipivlyt/assets


INFO:tensorflow:Assets written to: /tmp/tmpjipivlyt/assets
2023-11-22 23:29:52.484417: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2023-11-22 23:29:52.484466: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2023-11-22 23:29:52.522652: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: /tmp/tmpjipivlyt
2023-11-22 23:29:52.524392: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2023-11-22 23:29:52.524441: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: /tmp/tmpjipivlyt
2023-11-22 23:29:52.527273: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:388] MLIR V1 optimization pass is not enabled
2023-11-22 23:29:52.529080: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2023-11-22 23:29:52.966116: I tensorflow/cc/saved_model/loader.cc:217] Running initializatio

In [5]:
! ls -lh -g -o bees_wasps.tflite

-rw-rw-r-- 1 43M лис 22 23:29 bees_wasps.tflite


## Question 2

To be able to use this model, we need to know the index of the input and 
the index of the output. 

What's the output index for this model?

In [6]:
interpreter = tf.lite.Interpreter(model_path='bees_wasps.tflite')
interpreter.allocate_tensors()

input_index = interpreter.get_input_details()[0]['index']
output_index = interpreter.get_output_details()[0]['index']

input_index, output_index 

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.


(0, 13)

## Question 3

Now we need to turn the image into numpy array and pre-process it. 

> Tip: Check the previous homework. What was the pre-processing 
> we did there?

After the pre-processing, what's the value in the first pixel, the R channel?


In [7]:
URL = 'https://raw.githubusercontent.com/euthumos/mlcourse/main/09-serverless/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'
img = preprocessing.download_image(URL)
img = preprocessing.prepare_image(img, (150, 150))
X = preprocessing.preprocess_input(img)

In [8]:
first_pixel_rgb = X[0, 0, 0, :]
first_pixel_rgb

array([0.8901961 , 0.81960785, 0.7019608 ], dtype=float32)

## Question 4

Now let's apply this model to this image. What's the output of the model?

In [9]:
interpreter.set_tensor(input_index, X)
interpreter.invoke()
preds = interpreter.get_tensor(output_index)
preds

array([[0.14101154]], dtype=float32)

## Prepare the lambda code 

Now you need to copy all the code into a separate python file. You will 
need to use this file for the next two questions.

Tip: you can test this file locally with `ipython` or Jupyter Notebook 
by importing the file and invoking the function from this file.  


## Docker 

For the next two questions, we'll use a Docker image that we already 
prepared. This is the Dockerfile that we used for creating the image:

```docker
FROM public.ecr.aws/lambda/python:3.10
COPY bees-wasps-v2.tflite .
```

And pushed it to [`agrigorev/zoomcamp-bees-wasps:v2`](https://hub.docker.com/r/agrigorev/zoomcamp-bees-wasps/tags).

A few notes:

* The image already contains a model and it's not the same model
  as the one we used for questions 1-4.
* The version of Python is 3.10, so you need to use the right wheel for 
  TF-Lite. For Tensorflow 2.14.0, it's https://github.com/alexeygrigorev/tflite-aws-lambda/raw/main/tflite/tflite_runtime-2.14.0-cp310-cp310-linux_x86_64.whl


## Question 5

Download the base image `agrigorev/zoomcamp-bees-wasps:v2`. You can easily make it by using [docker pull](https://docs.docker.com/engine/reference/commandline/pull/) command.

So what's the size of this base image?

In [10]:
! docker pull agrigorev/zoomcamp-bees-wasps:v2

v2: Pulling from agrigorev/zoomcamp-bees-wasps
Digest: sha256:823f8536a45968f40ee3daf8a2da030b914912a382a4611610b3b84d36d2924c
Status: Image is up to date for agrigorev/zoomcamp-bees-wasps:v2
docker.io/agrigorev/zoomcamp-bees-wasps:v2


In [11]:
! docker images 

REPOSITORY                      TAG       IMAGE ID       CREATED      SIZE
agrigorev/zoomcamp-bees-wasps   v2        b9f6c13de368   4 days ago   662MB


## Question 6

Now let's extend this docker image, install all the required libraries
and add the code for lambda.

You don't need to include the model in the image. It's already included. 
The name of the file with the model is `bees-wasps-v2.tflite` and it's 
in the current workdir in the image (see the Dockerfile above for the 
reference).

Now run the container locally.

Score https://raw.githubusercontent.com/euthumos/mlcourse/main/09-serverless/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg

What's the output from the model?

In [12]:
! docker build -t bees-wasps-model .

[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.0s (0/0)                                    docker:desktop-linux
[+] Building 0.1s (0/1)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.1s
 => => transferring dockerfile: 32B                                        0.0s
[+] Building 0.3s (0/2)                                    docker:desktop-linux
 => [internal] load build definition from Dockerfile                       0.3s
 => => transferring dockerfile: 32B     

In [12]:
# ! docker run -it --rm -p 8080:8080 bees-wasps-model:latest

In [13]:
! docker images 

REPOSITORY                      TAG       IMAGE ID       CREATED             SIZE
bees-wasps-model                latest    7195346838d5   About an hour ago   778MB
agrigorev/zoomcamp-bees-wasps   v2        b9f6c13de368   4 days ago          662MB


In [14]:
url = 'http://localhost:8080/2015-03-31/functions/function/invocations'

data = {'url': 'https://raw.githubusercontent.com/euthumos/mlcourse/main/09-serverless/rtd9dhsmhwrdezeldzoqgijdg8a.jpeg'}

result = requests.post(url, json=data).json()
print(result)

[[0.35206058621406555]]
